In [4]:
import pandas as pd
import gensim
from gensim.models import Word2Vec, KeyedVectors
import nltk
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Todd
[nltk_data]     Howard\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from IPython.display import clear_output
import torch

def do_stuff(embeds_path, df_path):

    df = pd.read_csv(df_path, sep=';')
    sentences = list(map(nltk.word_tokenize, map(lambda x: x.lower(), df['text_content'].values)))
    # sentences[6][5:10]

    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit([[d] for d in df['category']])
    y = enc.transform([[d] for d in df['category']]).toarray()

    X = []

    X = torch.load(embeds_path).cpu().numpy()

    reports = []

    kf = KFold(n_splits=5)

    for i, (train_index, test_index) in enumerate(kf.split(X)):
        clf_kfold = MLPClassifier(random_state=1, max_iter=300).fit(X[train_index], y[train_index])
        reports.append(classification_report(
            y[test_index], clf_kfold.predict(X[test_index]), 
            target_names=enc.categories_[0], output_dict = True))
        
    total_result = dict()

    for k in reports[0].keys():
        vals = [dc[k] for dc in reports]

        metrics_dict = dict()
        for k2 in vals[0].keys():
            vals2 = [dc[k2] for dc in vals]
            avg_val = (sum(vals2) * 1.) / len(vals2)
            metrics_dict[k2] = avg_val

        total_result[k] = metrics_dict

    clear_output(wait=True)
    
    return pd.DataFrame(total_result).transpose()


In [6]:
out = do_stuff('data/tensor.pt',  'data/final_recipes.csv')
out.to_csv('outs/bert.csv')
out

,precision,recall,f1-score,support
ciasta i torty,0.525330,0.506229,0.477613,771.4
ciasteczka,0.468778,0.304960,0.356066,456.0
desery,0.559803,0.338690,0.397288,788.8
grill,0.399293,0.098884,0.150044,147.8
inne,0.006897,0.002564,0.003738,97.8
napoje i koktajle,0.614805,0.441012,0.498237,137.4
obiady,0.612825,0.550158,0.569543,1137.8
pieczywo,0.042857,0.022568,0.024735,34.0
przekąski,0.264637,0.054017,0.087083,480.4
przetwory,0.631514,0.316371,0.421121,234.4
